In [9]:
from math import *
import numpy as np
import matplotlib.pyplot as plt

Nodeと面は違うよね、  
温度はNodeごとに定義するんじゃない？じゃあ、Nodeの中にいくつかの面が取り込まれる方式になるはず、いやこれも違う、Nodeは面と独立  
Node間の関係はグラフ理論のやつ使ったらえんかな、データ構造的に工夫して格納しても良いけど  
Environmentはどうしよ、全体として  

基本的に熱平衡方程式は各ノード間、

In [8]:
#CubeSatを仮定
class Spacecraft:
    def __init__(self, node_num, size, tamperature, surface_char):
        self.node_number = node_num
        self.size = size #sizeは小さい順に入力
        self.temperature = temperature
        self.surface_characteristics = surface_char
        self.surface = {} #面の名前：面積，方向
        self.node = {} #nodeが温度、表面特性(α，ε)、各ノード間の温度伝達係数、形態係数
    #directionは小さい面から順にx,y,zとする
    def sc_area(self):
        l0, l1, l2 = self.size
        return (l0*l1, l0*l2, l1*l2)
    def ini_surface(self):
        area = sc_area(self)*2
        surface_name = ["px", "py", "pz", "mx", "my", "mz"]
        direction = [(1,0,0), (0,1,0), (0,0,1), (-1,0,0), (0,-1,0), (0,0,-1)]
        for i in range(len(surface_name)):
            self.surface[surface_name[i]] = {"area":area[i], "direction":direction[i], "surface_char":self.surface_characteristics[i]}
    def ini_node(self):
        self.node[] = {"temperature":self.temperature, }
        
class Enviromment:
    def __init__(self, ):
        

        
class Node:
    def __init__(self):
        
        

class Thermal_equation:
    def __init__(self, Node_i, other_Nodes):
        self.Node_i = Node_i
        self.Node_j = other_Nodes
        sel
        
    def Newton_method(self):
        
    
    def RungeKutta4(self):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-8-33eae08c3ef2>, line 22)

In [ ]:
node_num = 1 #number of Node
size = (12, 24, 36) #cm 6U
temperature = (25.0) #℃
surface_char = ()